In [8]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [9]:
from sklearn.pipeline import make_pipeline

In [10]:
! pip3 install mlflow

In [11]:
! mlflow --version

mlflow, version 2.18.0


In [12]:
import mlflow

In [13]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1733828272132, experiment_id='1', last_update_time=1733828272132, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [14]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [16]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [17]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2024/12/10 11:00:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run aged-frog-599 at: http://127.0.0.1:5000/#/experiments/1/runs/55125dffde68494692293605dc53fd29
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [19]:
from mlflow.tracking import MlflowClient

In [20]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '55125dffde68494692293605dc53fd29'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [39]:
path = '/workspaces/mlops-zoomcamp/04-orcherstration/web-service-mlflow/mlartifacts/1/55125dffde68494692293605dc53fd29/artifacts/model/model.pkl'
print(path)


/workspaces/mlops-zoomcamp/04-orcherstration/web-service-mlflow/mlartifacts/1/55125dffde68494692293605dc53fd29/artifacts/model/model.pkl


In [40]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [41]:
dv

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, min_samples_leaf=10,
                                       n_jobs=-1, random_state=0))])